Note: Pytorch模型训练实用教程（余霆嵩）
----
<https://github.com/tensor-yu/PyTorch_Tutorial>
Chapter4: 监控模型——可视化
---

**4.1 TensorBoardX**   
1. github:<https://github.com/lanpa/tensorboardX>   
2. API文档：<https://tensorboardPyTorch.readthedocs.io/en/latest/tutorial_zh.html#>
3. tensordoardX 提供13个函数，可以记录标量、图像、语音、文字等等，功能十分丰富。

**运行如下tensorboardX的官方demo.py**

In [1]:
# coding: utf-8
import os
import torch
import torchvision.utils as vutils
import numpy as np
import torchvision.models as models
from torchvision import datasets
from tensorboardX import SummaryWriter

resnet18 = models.resnet18(False)
writer = SummaryWriter(os.path.join("Result", "runs"))
sample_rate = 44100
freqs = [262, 294, 330, 349, 392, 440, 440, 440, 440, 440, 440]

true_positive_counts = [75, 64, 21, 5, 0]
false_positive_counts = [150, 105, 18, 0, 0]
true_negative_counts = [0, 45, 132, 150, 150]
false_negative_counts = [0, 11, 54, 70, 75]
precision = [0.3333333, 0.3786982, 0.5384616, 1.0, 0.0]
recall = [1.0, 0.8533334, 0.28, 0.0666667, 0.0]

In [2]:
for n_iter in range(100):
    s1 = torch.rand(1)  # value to keep
    s2 = torch.rand(1)
    # data grouping by `slash`
    writer.add_scalar(os.path.join("data", "scalar_systemtime"), s1[0], n_iter)
    # data grouping by `slash`
    writer.add_scalar(os.path.join("data", "scalar_customtime"), s1[0], n_iter, walltime=n_iter)
    writer.add_scalars(os.path.join("data", "scalar_group"), {"xsinx": n_iter * np.sin(n_iter),
                                             "xcosx": n_iter * np.cos(n_iter),
                                             "arctanx": np.arctan(n_iter)}, n_iter)
    x = torch.rand(32, 3, 64, 64)  # output from network
    if n_iter % 10 == 0:
        x = vutils.make_grid(x, normalize=True, scale_each=True)
        writer.add_image('Image', x, n_iter)  # Tensor
        # writer.add_image('astronaut', skimage.data.astronaut(), n_iter) # numpy
        # writer.add_image('imread',
        # skimage.io.imread('screenshots/audio.png'), n_iter) # numpy
        x = torch.zeros(sample_rate * 2)
        for i in range(x.size(0)):
            # sound amplitude should in [-1, 1]
            x[i] = np.cos(freqs[n_iter // 10] * np.pi *
                          float(i) / float(sample_rate))
        writer.add_audio('myAudio', x, n_iter)
        writer.add_text('Text', 'text logged at step:' + str(n_iter), n_iter)
        writer.add_text('markdown Text', '''a|b\n-|-\nc|d''', n_iter)
        for name, param in resnet18.named_parameters():
            if 'bn' not in name:
                writer.add_histogram(name, param, n_iter)
        writer.add_pr_curve('xoxo', np.random.randint(2, size=100), np.random.rand(
            100), n_iter)  # needs tensorboard 0.4RC or later
        writer.add_pr_curve_raw('prcurve with raw data', true_positive_counts,
                                false_positive_counts,
                                true_negative_counts,
                                false_negative_counts,
                                precision,
                                recall, n_iter)

In [5]:
# export scalar data to JSON for external processing
writer.export_scalars_to_json(os.path.join("Result", "all_scalars.json"))

dataset = datasets.MNIST(os.path.join( "Data", "mnist"), train=False, download=True)
images = dataset.test_data[:100].float()
label = dataset.test_labels[:100]
features = images.view(100, 784)
writer.add_embedding(features, metadata=label, label_img=images.unsqueeze(1))
writer.add_embedding(features, global_step=1, tag='noMetadata')
dataset = datasets.MNIST(os.path.join("Data", "mnist"), train=True, download=True)
images_train = dataset.train_data[:100].float()
labels_train = dataset.train_labels[:100]
features_train = images_train.view(100, 784)

all_features = torch.cat((features, features_train))
all_labels = torch.cat((label, labels_train))
all_images = torch.cat((images, images_train))
dataset_label = ['test'] * 100 + ['train'] * 100
all_labels = list(zip(all_labels, dataset_label))

writer.add_embedding(all_features, metadata=all_labels, label_img=all_images.unsqueeze(1),
                     metadata_header=['digit', 'dataset'], global_step=2)

9920512it [00:05, 1701730.89it/s]                                                                                      


Extracting Data\mnist\MNIST\raw\train-images-idx3-ubyte.gz to Data\mnist\MNIST\raw


32768it [00:00, 81357.61it/s]                                                                                          


Extracting Data\mnist\MNIST\raw\train-labels-idx1-ubyte.gz to Data\mnist\MNIST\raw


1654784it [00:01, 1469497.14it/s]                                                                                      


Extracting Data\mnist\MNIST\raw\t10k-images-idx3-ubyte.gz to Data\mnist\MNIST\raw


8192it [00:00, 17686.23it/s]                                                                                           


Extracting Data\mnist\MNIST\raw\t10k-labels-idx1-ubyte.gz to Data\mnist\MNIST\raw
Processing...
Done!


In [6]:
# VIDEO
vid_images = dataset.train_data[:16 * 48]
vid = vid_images.view(16, 1, 48, 28, 28)  # BxCxTxHxW

writer.add_video('video', vid_tensor=vid)
writer.add_video('video_1_fps', vid_tensor=vid, fps=1)

writer.close()

add_video needs package moviepy
add_video needs package moviepy
